In [1]:
import pandas as pd
from functions import pygeo_wrapper
from functions import data_preprocess_functions


In [ ]:
model = "t89"
filename = 'data/fulldata_20160101_to_20191231.csv'

In [2]:
data = data_preprocess_functions.preprocess_data(pd.read_csv(filename))
dispersion_list = data_preprocess_functions.extract_dispersions(data)